In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import re

In [18]:
import unicodedata

In [47]:
from tqdm import tqdm

In [73]:
from itertools import chain

In [2]:
nlp = spacy.load('ja_ginza')

In [3]:
train_df_raw = pd.read_csv("../data/train.txt", sep="\t")
val_df_raw = pd.read_csv("../data/validation.txt", sep="\t")

In [4]:
train_df_raw

,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Disgust,Reader3_Trust,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1,2012/07/31 23:48,train,0,1,2,1,1,0,...,1,0,0,2,0,0,0,0,0,0
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,1,2012/08/02 23:09,train,3,0,3,0,0,0,...,0,1,1,0,0,2,0,0,0,0
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1,2012/08/05 00:50,train,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,眠い、眠れない。,1,2012/08/08 01:36,train,0,2,1,0,0,1,...,2,0,0,1,0,0,0,0,1,0
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,1,2012/08/09 22:24,train,2,1,3,2,0,1,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,ジョージア移住系ブロガーになるって言っても、月間ボリューム100～1000かいな！,76,2019/10/26 16:55,train,0,2,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
39996,イギリスは6ヶ月未満、ビザなし滞在OK。凄い！,76,2019/10/26 23:24,train,0,0,0,3,0,0,...,0,0,1,0,0,2,0,0,0,0
39997,カナダは、6ヶ月以内の観光の場合、ビザなし滞在OK。凄い！,76,2019/10/26 23:26,train,0,0,0,3,0,0,...,0,0,1,0,0,2,0,0,0,0
39998,なぜ有料noteが重宝されているのかを解説したいが、需要がないので止めておこう。,76,2019/10/27 09:10,train,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [5]:
val_df_raw

,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Disgust,Reader3_Trust,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust
0,福岡に移住して、特に不満はない。\nけれど、慣れて来たので、他の街にも行ってみたくなってきて...,76,2019/10/27 10:46,train,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,うまくいった時の慢心には注意Σ(・∀・；),76,2019/10/27 10:52,train,0,0,0,0,0,2,...,0,0,0,0,0,0,0,1,0,0
2,お金持ちではなかったので、自分でカードゲームを作りだしたりして遊んでいた。\n\nその時の楽...,76,2019/10/27 11:23,train,0,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
3,飲み屋でしか聞けなかったような話って実はYouTubeで需要がある。,76,2019/10/27 15:11,train,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,①まずやってみる\n②疑問が出てきてから、その疑問解決のための教材を買う\n\n↑教材が高い...,76,2019/10/27 20:12,train,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,真夜中にふと思い立ち、ノートPCを持って部屋を出て、ダイニングで仕事したらすんごい捗った。\...,80,2020/09/15 08:01,train,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3196,ぐっどこんでぃしょん。\n心も頭もクリア。\n秋分の日のおかげかな？\n人と自然としっとり過...,80,2020/09/22 01:52,train,1,0,1,0,0,0,...,0,0,2,0,2,0,0,0,0,0
3197,朝から免許の更新へ。\n90分で終わり、出口へ向かうと献血の呼びかけが。\nみんな通り過ぎて...,80,2020/09/23 22:32,train,2,0,2,1,0,0,...,0,0,2,0,0,0,0,0,0,0
3198,夜も更けて参りましたが、食後のコーヒーが飲みたいのでドリップ開始…\n\nぼんやり秋の夜長を...,80,2020/10/11 00:12,train,2,0,1,0,0,0,...,0,0,0,0,2,0,0,0,0,0


In [6]:
text = "( ﾟдﾟ)!!!\n\nひばりやさんにきたら隣でズネーモン様がラーメン食べてた…！！(ﾟＡﾟ;)ｺﾞｸ"
[(token.text, np.linalg.norm(token.vector)) for token in nlp(text)]

[('(', 5.611514),
 ('ﾟ', 0.0),
 ('д', 3.5945606),
 ('ﾟ', 0.0),
 (')', 5.611514),
 ('!', 0.0),
 ('!', 0.0),
 ('!', 0.0),
 ('\n\n', 0.0),
 ('ひばり', 4.026902),
 ('や', 2.401923),
 ('さん', 2.496026),
 ('に', 1.8964515),
 ('き', 2.2270827),
 ('たら', 2.3955982),
 ('隣', 0.0),
 ('で', 1.8796002),
 ('ズネーモン', 0.0),
 ('様', 1.9903427),
 ('が', 1.8806041),
 ('ラーメン', 3.9732263),
 ('食べ', 0.0),
 ('て', 1.9703232),
 ('た', 1.912341),
 ('…', 2.8167915),
 ('！', 3.3613234),
 ('！', 3.3613234),
 ('(', 5.611514),
 ('ﾟ', 0.0),
 ('Ａ', 0.0),
 ('ﾟ', 0.0),
 (';', 0.0),
 (')', 5.611514),
 ('ｺﾞｸ', 0.0)]

In [7]:
def make_dataset(df_raw: pd.DataFrame):
    df = df_raw[["Sentence", "Writer_Joy"]]
    df.columns = ["text", "label"]
    return df

In [8]:
train_df = make_dataset(train_df_raw)
val_df = make_dataset(val_df_raw)

In [9]:
train_df

,text,label
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,0
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,3
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1
3,眠い、眠れない。,0
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,2
...,...,...
39995,ジョージア移住系ブロガーになるって言っても、月間ボリューム100～1000かいな！,0
39996,イギリスは6ヶ月未満、ビザなし滞在OK。凄い！,0
39997,カナダは、6ヶ月以内の観光の場合、ビザなし滞在OK。凄い！,0
39998,なぜ有料noteが重宝されているのかを解説したいが、需要がないので止めておこう。,0


In [10]:
val_df

,text,label
0,福岡に移住して、特に不満はない。\nけれど、慣れて来たので、他の街にも行ってみたくなってきて...,0
1,うまくいった時の慢心には注意Σ(・∀・；),0
2,お金持ちではなかったので、自分でカードゲームを作りだしたりして遊んでいた。\n\nその時の楽...,0
3,飲み屋でしか聞けなかったような話って実はYouTubeで需要がある。,0
4,①まずやってみる\n②疑問が出てきてから、その疑問解決のための教材を買う\n\n↑教材が高い...,0
...,...,...
3195,真夜中にふと思い立ち、ノートPCを持って部屋を出て、ダイニングで仕事したらすんごい捗った。\...,0
3196,ぐっどこんでぃしょん。\n心も頭もクリア。\n秋分の日のおかげかな？\n人と自然としっとり過...,1
3197,朝から免許の更新へ。\n90分で終わり、出口へ向かうと献血の呼びかけが。\nみんな通り過ぎて...,2
3198,夜も更けて参りましたが、食後のコーヒーが飲みたいのでドリップ開始…\n\nぼんやり秋の夜長を...,2


In [11]:
train_df.to_csv("../data/train.tsv", sep="\t", index=False)
val_df.to_csv("../data/test.tsv", sep="\t", index=False)

In [17]:
[np.linalg.norm(token.vector) for token in nlp("youtube")]

[0.0]

In [14]:
train_df

,text,label
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,0
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,3
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1
3,眠い、眠れない。,0
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,2
...,...,...
39995,ジョージア移住系ブロガーになるって言っても、月間ボリューム100～1000かいな！,0
39996,イギリスは6ヶ月未満、ビザなし滞在OK。凄い！,0
39997,カナダは、6ヶ月以内の観光の場合、ビザなし滞在OK。凄い！,0
39998,なぜ有料noteが重宝されているのかを解説したいが、需要がないので止めておこう。,0


In [20]:
def preprocess_text(text: str) -> str:
  text = unicodedata.normalize("NFKC", text)
  text = re.sub(r"\s+", " ", text)
  text = text.lower()
  return text

In [21]:
preprocess_text("（笑）")

'(笑)'

In [22]:
preprocess_text("うふ、うふふふふふふふふ\(//∇//)\ あははははははははははは(⌒▽⌒) えへへへへへへへへへへへ(*´∀｀*)")

'うふ、うふふふふふふふふ\\(//∇//)\\ あははははははははははは(⌒▽⌒) えへへへへへへへへへへへ(* ́∀`*)'

In [23]:
def show_word2vec(text: str) -> None:
    doc = nlp(text)
    for token in doc:
        print(token.text, np.linalg.norm(token.vector))

In [27]:
text = "長野大学きたけど駐車場が結構いっぱいだなー( ?????;)\n構内通り抜けて一番上の砂利のとこはまだ停めれる??\(´ω` )/????\(　´)/????\( ´ω`)/??"
show_word2vec(preprocess_text(text))

長野大学 4.1560984
き 2.2270827
た 1.912341
けど 0.0
駐車場 3.8262544
が 1.8806041
結構 2.7068055
いっぱい 0.0
だ 1.9963483
なー 0.0
( 5.611514
? 0.0
? 0.0
? 0.0
? 0.0
? 0.0
; 0.0
) 5.611514
構内 3.8007216
通り抜け 4.186805
て 1.9703232
一番 2.7271662
上 2.5119755
の 1.8054866
砂利 3.802324
の 1.8054866
とこ 2.324408
は 1.8471361
まだ 0.0
停め 0.0
れる 2.2392254
? 0.0
? 0.0
\ 0.0
( 5.611514
́ω 0.0
` 0.0
) 5.611514
/ 3.0649598
? 0.0
? 0.0
? 0.0
? 0.0
\ 0.0
( 5.611514
 ́ 0.0
) 5.611514
/ 3.0649598
? 0.0
? 0.0
? 0.0
? 0.0
\ 0.0
( 5.611514
́ω 0.0
` 0.0
) 5.611514
/ 3.0649598
? 0.0
? 0.0


In [31]:
nlp.vocab.vectors.data.shape

(20000, 300)

In [32]:
nlp.vocab.strings["長野大学"]

6174798505746749661

In [34]:
np.linalg.norm(nlp.vocab.vectors[nlp.vocab.strings["長野大学"]])

4.1560984

In [35]:
nlp.vocab.vectors.key2row[6174798505746749661]

18394

In [36]:
text = "長野大学きたけど駐車場が結構いっぱいだなー( ?????;)\n構内通り抜けて一番上の砂利のとこはまだ停めれる??\(´ω` )/????\(　´)/????\( ´ω`)/??"
doc = nlp(preprocess_text(text))

In [44]:
doc[0].pos_

'PROPN'

In [45]:
for token in doc:
  print(token.text, token.pos_)

長野大学 PROPN
き VERB
た AUX
けど SCONJ
駐車場 NOUN
が ADP
結構 ADV
いっぱい ADJ
だ AUX
なー PART
( PUNCT
? PUNCT
? PUNCT
? PUNCT
? PUNCT
? PUNCT
; SYM
) PUNCT
構内 NOUN
通り抜け VERB
て SCONJ
一番 ADV
上 NOUN
の ADP
砂利 NOUN
の ADP
とこ NOUN
は ADP
まだ ADV
停め VERB
れる AUX
? PUNCT
? PUNCT
\ NOUN
( PUNCT
́ω NOUN
` NUM
) PUNCT
/ SYM
? PUNCT
? PUNCT
? PUNCT
? PUNCT
\ NOUN
( PUNCT
 ́ SYM
) PUNCT
/ SYM
? PUNCT
? PUNCT
? PUNCT
? PUNCT
\ NOUN
( PUNCT
́ω NOUN
` NUM
) PUNCT
/ SYM
? PUNCT
? PUNCT


In [50]:
n_token = 0
n_token_has_vector = 0
exclude_pos = ["PUNCT", "SYM"]
vector_not_found = set()
for text in tqdm(train_df["text"]):
  doc = nlp(preprocess_text(text))
  for token in doc:
    if token.pos_ not in exclude_pos:
      n_token += 1
      if token.has_vector:
        n_token_has_vector += 1
      else:
        vector_not_found.add(token.text)

n_token_has_vector / n_token

100%|██████████| 40000/40000 [09:19<00:00, 71.45it/s]   


0.8225313448987519

In [49]:
n_token - n_token_has_vector

143003

In [51]:
vector_not_found

{'ケロっと',
 'エアリズムマスク',
 'よっぽど',
 'すれ違い',
 'なさん',
 '踏ま',
 '押しかけ',
 '持続化',
 '響け',
 '想い出',
 'ふうう',
 'お祝い',
 '指値',
 'どーぞ',
 '補整',
 'ぜれ',
 'あああ',
 'ヒトガタ',
 'スゲェ',
 '掘ら',
 '片脚',
 '弱く',
 '断た',
 'とびっきり',
 'バグれ',
 '縮まっ',
 'ペンクソ',
 '未就学児',
 'ジャパリケット',
 'はやぶさ',
 'フルコートバスケ',
 'rom',
 '紡げ',
 'ねだる',
 '新型コロナ',
 '目ぇ',
 'クラフトピア',
 '汲ん',
 'バレエプロ',
 'がんばっ',
 '妬く',
 'からっぽ',
 'イビキ',
 'linux',
 'スズメ',
 'blu-ray',
 'てて',
 'いもうと',
 'ごとく',
 '刈ら',
 'バズっ',
 '右衛門',
 '仕掛かり',
 '取り逃し',
 'たい焼き',
 'microsoft',
 'ささっ',
 '似合っ',
 'マイナンバーカード',
 'ぶっ叩い',
 '訳せる',
 'icカードリーダー',
 '(●`',
 'きな粉',
 'トガリ',
 'クラシックプロ',
 '寝れる',
 '持ち変え',
 '組ん',
 'デカかっ',
 '生きれる',
 '田んぼ',
 'お疲れさま',
 'よーーーー',
 'ありきたり',
 'バレンティン',
 'ジェイソンベルモンテ',
 '聞き返せ',
 '黒かっ',
 '願っ',
 'まくら',
 'カニミソバーニャカウダ',
 'ジッポー',
 'まぁまぁ',
 '打とう',
 'おおー',
 '美し',
 'フーフー',
 'こまけえ',
 '生ん',
 'めちゃ',
 'ココイチ',
 'instagram',
 '下ごしらえ',
 '曇ら',
 '予備試験',
 '待てん',
 'アアアアアアアアアアアアアアアアアアアアアアアアアアアアアアアアアアイラアアアアアアアアアアアアアア',
 'テイストオブカオス',
 '喰らう',
 'ガルパ',
 'カクカク',
 'あがれ',
 '過ごせる',
 'オングロ',
 'イジメ',
 '聞き入れ',

In [52]:
len(vector_not_found)

13103

In [53]:
from janome.tokenizer import Tokenizer

In [54]:
tokenizer = Tokenizer()

In [55]:
[token.surface for token in tokenizer.tokenize("ob訪問")]

['ob', '訪問']

In [62]:
nlp.vocab["ネタばれ"].has_vector

False

In [59]:
nlp

In [64]:
nlp.vocab.set_vector("@pad@", np.zeros(300))
nlp.vocab.set_vector("@unk@", np.zeros(300))

In [66]:
nlp.vocab["@pad@"].rank

20000

In [67]:
nlp.vocab.vectors.data.shape

(26000, 300)

In [70]:
def text2seq(text: str, padded_len: int) -> np.ndarray:
  doc = nlp(preprocess_text(text))
  assert len(doc) <= padded_len
  seq = np.zeros(padded_len, dtype=np.int32)
  head = padded_len - len(doc)
  for i, token in enumerate(doc):
    if token.has_vector:
      seq[head + i] = token.rank
    else:
      seq[head + i] = nlp.vocab["@unk@"].rank
  return seq

In [71]:
text = "長野大学きたけど駐車場が結構いっぱいだなー( ?????;)\n構内通り抜けて一番上の砂利のとこはまだ停めれる??\(´ω` )/????\(　´)/????\( ´ω`)/??"
text2seq(text, 160)

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0, 18394,   391,     5, 20001,  3188,     6,   331, 20001,
           4, 20001,  7181, 20001, 20001, 20001, 20001, 20001, 20001,
        7181, 12131,  7594,     2,   257,   151,     0, 16648,     0,
         764,     7,

In [74]:
max_len = 0
for text in tqdm(chain(train_df["text"], val_df["text"])):
  text = preprocess_text(text)
  doc = nlp(text)
  max_len = max(max_len, len(doc))

max_len

43200it [18:43, 38.47it/s] 


155

In [75]:
nlp.vocab.vectors.data[0]

array([-8.27400386e-02, -9.10336450e-02, -8.74446332e-02, -1.43936828e-01,
       -5.31590357e-02,  9.33098793e-02, -5.67257181e-02, -7.48155788e-02,
       -6.20616972e-02, -9.27804783e-02, -2.83585321e-02, -3.02598067e-02,
        5.98407239e-02,  3.00214812e-02, -1.64748237e-01, -2.61503696e-01,
       -1.57922238e-01, -7.42753297e-02, -1.31985053e-01, -2.92436760e-02,
       -1.42266735e-01,  1.95084319e-01, -9.21938941e-03,  3.29293609e-02,
       -1.39554635e-01, -1.36195734e-01, -1.94662690e-01, -1.80367772e-02,
        5.22504486e-02,  5.67570440e-02,  1.48480758e-02, -5.34212999e-02,
        1.13662310e-01,  7.46132731e-02,  4.00496498e-02, -6.36053532e-02,
       -9.86220166e-02,  1.28792807e-01, -1.83209032e-01,  7.17505813e-02,
        1.26726344e-01,  1.12625964e-01, -1.26824796e-01, -9.32623893e-02,
       -4.37840000e-02, -1.78249627e-01, -5.04186843e-03, -7.68889487e-03,
       -2.22529575e-01,  7.30498508e-02, -2.76429206e-02,  3.77984010e-02,
        1.09656202e-02,  

In [76]:
show_word2vec("\( ´ω`)/")

\ 0.0
( 5.611514
´ 3.6753085
ω 3.630061
` 0.0
) 5.611514
/ 3.0649598
